In [ ]:
import os
import django

# os.path.abspath(os.curdir) # notebooks
os.chdir('../src')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "core.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [ ]:
from abcdef.helpers import process_files_all
#process_files_all()


In [ ]:
import pandas as pd
from psycopg2.extras import execute_values
from abcdef import models as abcdef_models
from abcdef.helpers import process_file, process_files_all, batch_update

fn3 = '/home/www/python/examples/atruck/files/ABC-3xx.csv'
fn4 = '/home/www/python/examples/atruck/files/ABC-4xx.csv'
fn8 = '/home/www/python/examples/atruck/files/ABC-8xx.csv'
fn9 = '/home/www/python/examples/atruck/files/DEF-9xx.csv'

df = pd.read_csv(fn9, sep=';', dtype=str)
df = df.set_axis('АВС/ DEF;От;До;Емкость;Оператор;Регион;Территория ГАР;ИНН'.split(';'), axis='columns')
pred = [{'Оператор-ID': o.id, 'Оператор': o.name} for o in abcdef_models.Opsos.objects.all()]
df_opsos = pd.DataFrame(pred)
pred = [{'Территория ГАР-ID': o.id, 'Территория ГАР': o.name} for o in abcdef_models.Territory.objects.all()]
df_ter = pd.DataFrame(pred)

df_norm = df['АВС/ DEF;От;До;Емкость;Оператор;Территория ГАР;ИНН'.split(';')]
df_norm = pd.merge(df_norm, df_opsos, how='left', on='Оператор')
df_norm = pd.merge(df_norm, df_ter, how='left', on='Территория ГАР')
df_norm = df_norm['АВС/ DEF;От;До;Емкость;Оператор-ID;Территория ГАР-ID;ИНН'.split(';')]

df_norm['num_min'] = df_norm['АВС/ DEF'] + df_norm['От']
df_norm['num_max'] = df_norm['АВС/ DEF'] + df_norm['До']
df_norm = df_norm.astype({'num_max': str, 'num_min': str})

df_norm



# df_ter = df[['Территория ГАР']].drop_duplicates()
# batch_update(
#     table_name=abcdef_models.Territory._meta.db_table,
#     data=df_ter.to_records(index=False).tolist(),
#     col_keys=['name'],
#     col_upd=['name'],
#     col_names=['name'],
# )

# df_ter = df['Территория ГАР'].drop_duplicates()
# df_norm = df['АВС/ DEF;От;До;Емкость;Оператор;Регион;Территория ГАР;ИНН'.split(';')]

# df_norm

,АВС/ DEF,От,До,Емкость,Оператор-ID,Территория ГАР-ID,ИНН,num_min,num_max
0,900,0000000,0061999,62000,1,1,7743895280,9000000000,9000061999
1,900,0062000,0062999,1000,1,2,7743895280,9000062000,9000062999
2,900,0063000,0099999,37000,1,1,7743895280,9000063000,9000099999
3,900,0100000,0199999,100000,2,3,7743895280,9000100000,9000199999
4,900,0200000,0299999,100000,2,4,7743895280,9000200000,9000299999
...,...,...,...,...,...,...,...,...,...
16414,999,9440000,9479999,40000,19,46,9705121882,9999440000,9999479999
16415,999,9480000,9489999,10000,4,7,7707840631,9999480000,9999489999
16416,999,9490000,9499999,10000,31,84,7812014560,9999490000,9999499999
16417,999,9500000,9599999,100000,4,7,7707840631,9999500000,9999599999
